<a href="https://colab.research.google.com/github/C-Little26/ECGR4105/blob/main/Hw5/ECGR4105_Hw5_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#necessary libraries
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import torch
import tensorflow
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
filepath = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(filepath))
housing.isnull().sum()*100/housing.shape[0] #use to check for null values

price               0.0
area                0.0
bedrooms            0.0
bathrooms           0.0
stories             0.0
mainroad            0.0
guestroom           0.0
basement            0.0
hotwaterheating     0.0
airconditioning     0.0
parking             0.0
prefarea            0.0
furnishingstatus    0.0
dtype: float64

In [3]:
#variables used for this model
varlist =  ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
#establish matrices
Y = housing.pop('price')
X = housing[varlist]
#train/test split
x_train, x_val, y_train, y_val = train_test_split(X,Y, train_size = 0.8, test_size = 0.2, random_state=0)

In [4]:
#standardize values
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_val = sc_x.fit_transform(x_val)

In [5]:
#stack arrays before conversion
x_t_arr = np.c_[np.ones((len(y_train), 1)), x_train]
x_v_arr = np.c_[np.ones((len(y_val),1)), x_val]

#convert to tensors
x_t_ten = torch.tensor(x_t_arr)
x_v_ten = torch.tensor(x_v_arr)
y_t_ten = torch.tensor(y_train.values)
y_v_ten = torch.tensor(y_val.values)
#have to call .values for it to be treated like an array

In [6]:
#define the model
def model(t_u, w5, w4, w3, w2, w1, b):
  return w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b

#linear model to test something out later
def linear(t_u, w, b):
    return w * t_u + b

#define the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [7]:
#define the training loop
def loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:  # <1>
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params

In [8]:
# Linear training loop, yet again for testing something later
def linear_loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
    for epoch in range(1, n_epochs+1):
        t_p_train = linear(t_u_train, *params)
        loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

        t_p_val = linear(t_u_val, *params)
        loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
    return params

In [9]:
#establish variables for model iterations
learning_rate = [0.00001, 0.000001, 0.0000001, 0.00000001]
sgd_params = torch.zeros(len(learning_rate), 6)
sgd_loss = torch.zeros(len(learning_rate), 1)
#run training for each learning rate using SGD
for i in learning_rate:
    params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
    optimizer = optim.SGD([params], lr=i)
    print(f'\nTraining with learning rate: {i}')
    sgd_params[learning_rate.index(i)] = loop(
        n_epochs = 5000, optimizer = optimizer,
        params = params,
        t_u_train = x_t_ten, t_u_val = x_v_ten,
        t_c_train = y_t_ten, t_c_val = y_v_ten, )


Training with learning rate: 1e-05
Epoch 500, Training loss: 24062939433895.2656, Validation loss: 22573483960989.0234

Epoch 1000, Training loss: 22756123929035.6523, Validation loss: 21339278009590.9805

Epoch 1500, Training loss: 21843127180528.6367, Validation loss: 20509121276043.1250

Epoch 2000, Training loss: 21155111802891.4375, Validation loss: 19897552080648.2695

Epoch 2500, Training loss: 20597387184874.6562, Validation loss: 19403807486485.5156

Epoch 3000, Training loss: 20116806891972.0781, Validation loss: 18973722858142.0977

Epoch 3500, Training loss: 19683494287205.6680, Validation loss: 18578605914666.5156

Epoch 4000, Training loss: 19280615772127.4688, Validation loss: 18203574011512.0859

Epoch 4500, Training loss: 18898644426215.6875, Validation loss: 17841143001456.6250

Epoch 5000, Training loss: 18532149408804.8633, Validation loss: 17487723404393.4121


Training with learning rate: 1e-06
Epoch 500, Training loss: 25816266826436.4805, Validation loss: 24300

In [10]:
#establish variables for model iterations
learning_rate = [0.00001, 0.000001, 0.0000001, 0.00000001]
adam_params = torch.zeros(len(learning_rate), 6)
adam_loss = torch.zeros(len(learning_rate), 1)
#run training for each learning rate using SGD
for i in learning_rate:
    params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
    optimizer = optim.Adam([params], lr=i)
    print(f'\nTraining with learning rate: {i}')
    sgd_params[learning_rate.index(i)] = loop(
        n_epochs = 5000, optimizer = optimizer,
        params = params,
        t_u_train = x_t_ten, t_u_val = x_v_ten,
        t_c_train = y_t_ten, t_c_val = y_v_ten, )


Training with learning rate: 1e-05
Epoch 500, Training loss: 26469775977544.0156, Validation loss: 25188977638937.3398

Epoch 1000, Training loss: 26469774913841.3086, Validation loss: 25188976266172.1680

Epoch 1500, Training loss: 26469773850140.7656, Validation loss: 25188974893409.2773

Epoch 2000, Training loss: 26469772786437.2383, Validation loss: 25188973520643.6016

Epoch 2500, Training loss: 26469771722734.0547, Validation loss: 25188972147878.4023

Epoch 3000, Training loss: 26469770659031.7734, Validation loss: 25188970775114.2500

Epoch 3500, Training loss: 26469769595337.0703, Validation loss: 25188969402357.7422

Epoch 4000, Training loss: 26469768531645.4922, Validation loss: 25188968029604.4766

Epoch 4500, Training loss: 26469767467954.8242, Validation loss: 25188966656852.2617

Epoch 5000, Training loss: 26469766404265.0586, Validation loss: 25188965284101.0898


Training with learning rate: 1e-06
Epoch 500, Training loss: 26469776937791.3594, Validation loss: 25188

In [14]:
#train with a linear model?
#learning_rate = [0.00001, 0.000001, 0.0000001, 0.00000001]
#lin_params = torch.zeros(len(learning_rate), 2)
#lin_loss = torch.zeros(len(learning_rate), 1)
#run training for each learning rate using linear model
#for i in learning_rate:
#    params = torch.tensor([1.0, 0.0], requires_grad = True)
#    optimizer = optim.SGD([params], lr=i)
#    print(f'\nTraining with learning rate: {i}')
#    sgd_params[learning_rate.index(i)] = linear_loop(
#       n_epochs = 5000, optimizer = optimizer,
#       params = params,
 #       t_u_train = x_t_ten, t_u_val = x_v_ten,
 #       t_c_train = y_t_ten, t_c_val = y_v_ten, )

#doesn't work